In [100]:
# Cell 1: Imports and Configuration
import os
import json
import webbrowser
import time
from pathlib import Path
from urllib.parse import urlparse, parse_qs
from datetime import datetime, timedelta, timezone
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import WebApplicationClient, TokenExpiredError
from dotenv import load_dotenv

print("--- Cell 1: Loading Configuration ---")

# Load .env file
load_dotenv(dotenv_path='./.env', override=True)

# --- Configuration Variables ---
XERO_CLIENT_ID = os.getenv("XERO_CLIENT_ID")
XERO_CLIENT_SECRET = os.getenv("XERO_CLIENT_SECRET")
XERO_REDIRECT_URI = os.getenv("XERO_REDIRECT_URI")
TOKEN_FILE_PATH = Path(os.getenv("XERO_TOKEN_FILE_PATH", "./xero_tokens.json"))

XERO_AUTHORIZATION_BASE_URL = "https://login.xero.com/identity/connect/authorize"
XERO_TOKEN_URL = "https://identity.xero.com/connect/token"
XERO_CONNECTIONS_URL = "https://api.xero.com/connections"
XERO_API_BASE_URL = "https://api.xero.com/api.xro/2.0"
XERO_SCOPES = [
    "openid", "profile", "email", "offline_access",
    "accounting.transactions", "accounting.contacts", "accounting.settings.read",
    # Ensure accounting.transactions.read or similar is included if needed
]

# --- Validate Configuration ---
if not all([XERO_CLIENT_ID, XERO_CLIENT_SECRET, XERO_REDIRECT_URI]):
    raise ValueError("Error: Missing required Xero environment variables in .env file.")
else:
    print("✅ Configuration loaded successfully.")
    print(f"   Token File Path: {TOKEN_FILE_PATH.resolve()}") # Show absolute path

--- Cell 1: Loading Configuration ---
✅ Configuration loaded successfully.
   Token File Path: /Users/leon-ticharwa/Documents/repos/ai-agent/notebook/xero_tokens.json


In [101]:
# Cell 2: Utility Functions (Load/Save Tokens, Get Tenant ID, Ensure Token Validity, Get Accounts, Fetch Page)

# --- Token Load/Save ---
def load_tokens() -> dict | None:
    """Loads token data from the local JSON file."""
    if not TOKEN_FILE_PATH.exists(): print(f"Token file '{TOKEN_FILE_PATH}' not found."); return None
    try:
        with open(TOKEN_FILE_PATH, 'r') as f:
            token_data = json.load(f)
            if 'access_token' in token_data: print(f"✅ Tokens loaded from '{TOKEN_FILE_PATH}'."); return token_data
            else: print("⚠️ Loaded file missing access_token."); return None
    except Exception as e: print(f"Error loading tokens: {e}"); return None

def save_tokens(token_data: dict) -> None:
    """Saves token data to the local JSON file, adding expiry."""
    try:
        if 'expires_in' in token_data: token_data['expires_at'] = time.time() + int(token_data.get('expires_in', 0))
        with open(TOKEN_FILE_PATH, 'w') as f: json.dump(token_data, f, indent=2); print(f"✅ Tokens saved to '{TOKEN_FILE_PATH}'.")
    except Exception as e: print(f"Error saving tokens: {e}")

# --- Get Tenant ID ---
def get_xero_tenant_id(access_token: str) -> str | None:
    """Fetches the connected Xero Tenant ID using the access token."""
    if not access_token: return None
    print(f"\nFetching Tenant ID...")
    try:
        response = requests.get(XERO_CONNECTIONS_URL, headers={'Authorization': f'Bearer {access_token}', 'Accept': 'application/json'}, timeout=10)
        response.raise_for_status()
        connections = response.json()
        if connections and len(connections) > 0: tenant_id = connections[0].get('tenantId'); print(f"✅ Found Xero Tenant ID: {tenant_id}"); return tenant_id
        print("⚠️ Tenant ID not found in connection details."); return None
    except Exception as e: print(f"❌ Error fetching Tenant ID: {e}"); return None

# --- Ensure Token Validity (with Refresh Logic) ---
def ensure_valid_token(token_data: dict | None) -> dict | None:
    """Checks token validity and attempts refresh if needed. Returns valid token data or None."""
    if not token_data: return None
    is_expired = token_data.get('expires_at', 0) < time.time() - 60
    if not is_expired and 'access_token' in token_data: print("   Token valid."); return token_data
    if 'refresh_token' not in token_data: print("   ❌ Token expired/invalid, no refresh token."); return None
    print("   Token expired/expiring, attempting refresh...");
    try:
        refresh_session = OAuth2Session(XERO_CLIENT_ID, token=token_data)
        new_token_data = refresh_session.refresh_token(
            XERO_TOKEN_URL,
            refresh_token=token_data['refresh_token'],
            client_id=XERO_CLIENT_ID,
            client_secret=XERO_CLIENT_SECRET
        )
        print("   ✅ Token refreshed successfully.")
        save_tokens(new_token_data)
        return new_token_data
    except TokenExpiredError: print("   ❌ Refresh token itself expired/revoked."); return None
    except Exception as e: print(f"   ❌ Refresh failed: {e}"); return None

# --- Get Bank Accounts ---
def get_active_bank_accounts(access_token: str, tenant_id: str) -> list:
    """Fetches active bank accounts from Xero."""
    print(f"\nFetching active bank accounts...")
    bank_accounts = []
    accounts_url = f"{XERO_API_BASE_URL}/Accounts"
    headers = {'Authorization': f'Bearer {access_token}', 'Xero-Tenant-Id': tenant_id, 'Accept': 'application/json'}
    params = {'where': 'Type=="BANK" AND Status=="ACTIVE"'}
    try:
        response = requests.get(accounts_url, headers=headers, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        fetched_accounts = data.get('Accounts', [])
        bank_accounts = [{'AccountID': acc.get('AccountID'), 'Name': acc.get('Name')}
                         for acc in fetched_accounts if acc.get('AccountID') and acc.get('Name')]
        print(f"✅ Found {len(bank_accounts)} active bank account(s).")
    except Exception as e:
        print(f"   ❌ Error fetching Xero Accounts: {e}")
        if hasattr(e, 'response') and e.response is not None:
             try: print(f"      Status: {e.response.status_code}, Body: {e.response.text[:500]}...")
             except Exception: pass
    return bank_accounts

# --- Fetch Single Page of Bank Transactions ---
def get_xero_bank_transactions_page(access_token: str, tenant_id: str, bank_account_id: str, if_modified_since: str | None = None, page: int = 1):
    """Fetches a single page of bank transactions for a specific account."""
    bank_transactions_url = f"{XERO_API_BASE_URL}/BankTransactions"
    where_filter = f'BankAccount.AccountID==GUID("{bank_account_id}")'
    # print(f"      Fetching page {page} for Acct {bank_account_id}...") # Verbose log
    headers = {'Authorization': f'Bearer {access_token}', 'Xero-Tenant-Id': tenant_id, 'Accept': 'application/json'}
    if if_modified_since: headers['If-Modified-Since'] = if_modified_since
    params = {'page': page, 'pageSize': 100, 'where': where_filter}
    try:
        response = requests.get(bank_transactions_url, headers=headers, params=params, timeout=45)
        if response.status_code == 304: print(f"      -> No modifications for Acct {bank_account_id}."); return []
        response.raise_for_status()
        data = response.json()
        return data.get('BankTransactions', [])
    except requests.exceptions.RequestException as e:
        print(f"      ❌ Error fetching Bank Tx page {page} for Acct {bank_account_id}: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try: print(f"         Status: {e.response.status_code}, Body: {e.response.text[:500]}...")
            except Exception: pass
        return None # Indicate error

print("✅ Utility functions defined.")

✅ Utility functions defined.


In [104]:
# Cell 4: Fetch ALL Bank Transactions into a Single List and Print Full Details

print("\n--- Cell 4: Fetching ALL Bank Transactions Per Account ---")

# Make sure necessary imports are available
import json
import time
import requests
# Assumes utility functions (ensure_valid_token, get_xero_bank_transactions_page)
# and variables (valid_token_data, tenant_id, bank_accounts, XERO_API_BASE_URL)
# are available from previously executed cells (Cell 2 & Cell 3).

# --- Initialize a SINGLE list to store ALL results ---
all_bank_transactions_list = []
grand_total_fetched = 0

# Check necessary variables exist from Cell 3
if 'valid_token_data' not in globals() or not valid_token_data or not valid_token_data.get('access_token') \
   or 'tenant_id' not in globals() or not tenant_id \
   or 'bank_accounts' not in globals() or not isinstance(bank_accounts, list):
    print("❌ Cannot proceed. Auth data or bank account list missing/invalid from Cell 3.")
    print("   Please ensure Cell 3 executed successfully.")
elif not bank_accounts:
    print("ℹ️ No active bank accounts were found in Cell 3 to fetch transactions for.")
else:
    # Use the variables established in Cell 3
    current_token_data_for_fetch = valid_token_data.copy() # Work with a copy
    current_tenant_id = tenant_id
    current_bank_accounts = bank_accounts

    if_modified_since_str = None # Fetch all initially

    # Initial token check
    current_token_data_for_fetch = ensure_valid_token(current_token_data_for_fetch) # Assumes ensure_valid_token is defined
    if not current_token_data_for_fetch or not current_token_data_for_fetch.get('access_token'):
         print("❌ Initial token check failed. Cannot fetch.")
    else:
        # --- Loop Through Each Bank Account ---
        for account in current_bank_accounts:
            # Optional: Refresh check before starting each account
            # current_token_data_for_fetch = ensure_valid_token(current_token_data_for_fetch)
            # if not current_token_data_for_fetch or not current_token_data_for_fetch.get('access_token'):
            #     print(f"❌ Token became invalid before processing account {account.get('Name')}. Stopping.")
            #     break

            access_token = current_token_data_for_fetch['access_token']
            account_id = account.get('AccountID')
            account_name = account.get('Name', f'Unknown ({account_id})')

            if not account_id:
                 print(f"⚠️ Skipping account entry with missing ID: {account}")
                 continue

            print(f"\n--- Processing Account: {account_name} ({account_id}) ---")
            current_page = 1
            processed_in_account = 0

            # Loop to fetch pages for the current account
            while True:
                # Optional: Refresh check within page loop if very long running
                # ...

                bank_transactions_page = get_xero_bank_transactions_page( # Assumes this function is defined
                    access_token, current_tenant_id, account_id,
                    if_modified_since=if_modified_since_str, page=current_page
                )

                if bank_transactions_page is None: # API Error occurred
                    print(f"   Stopping fetch for {account_name} due to API error on page {current_page}.")
                    break # Exit the inner while loop for this account

                processed_in_account += len(bank_transactions_page)

                # Add ALL transactions from the page to the single list
                if bank_transactions_page:
                    print(f"      Fetched {len(bank_transactions_page)} transactions on page {current_page}.")
                    for tx in bank_transactions_page:
                        # Add account info for context before adding to the main list
                        tx['_account_name'] = account_name
                        tx['_account_id'] = account_id
                    all_bank_transactions_list.extend(bank_transactions_page) # Extend the main list

                # Decide whether to fetch next page
                if len(bank_transactions_page) < 100: # Assumes page size is 100 max
                    if current_page == 1 and not bank_transactions_page:
                         print(f"   No bank transactions found for {account_name} (page 1).")
                    else:
                        print(f"   Reached last page for {account_name} (fetched {len(bank_transactions_page)}).")
                    break # Stop fetching pages for this account
                else:
                    # Continue to the next page
                    current_page += 1
                    print(f"   Fetching next page ({current_page}) for {account_name}...")
                    time.sleep(0.5) # API kindness
                # End of page processing

            grand_total_fetched += processed_in_account # Add account total to grand total
            print(f"--- Finished Account: {account_name}. Processed {processed_in_account} total transactions. ---")
            # End of account loop

    # --- Process/Display ALL Fetched Transactions ---
    print("\n==========================================================")
    print(f"--- Total ALL Bank Transactions Fetched: {grand_total_fetched} ---")
    # Note: len(all_bank_transactions_list) should equal grand_total_fetched if no errors occurred mid-fetch
    print(f"--- (Collected {len(all_bank_transactions_list)} transactions in the list) ---")
    print("==========================================================")

    if not all_bank_transactions_list:
        print("\nNo bank transactions were fetched successfully.")
    else:
        # Sort the single list by date if desired (descending)
        all_bank_transactions_list.sort(key=lambda x: x.get('DateString', ''), reverse=True)

        print("\n--- DETAILS of ALL Fetched Bank Transactions (JSON) ---")
        # Iterate through the single combined list
        for i, tx in enumerate(all_bank_transactions_list):
             print(f"\n--- Fetched Tx {i+1} (Account: {tx.get('_account_name', 'N/A')}) ---")
             # Use json.dumps for pretty printing the entire object
             print(json.dumps(tx, indent=4)) # indent=4 for readability

        # Optional: Save the combined list to file if needed
        # output_filename = "xero_bank_transactions_all.json"
        # try:
        #     with open(output_filename, 'w') as f: json.dump(all_bank_transactions_list, f, indent=2)
        #     print(f"\nSaved all transactions to {output_filename}")
        # except Exception as e: print(f"\nError saving file: {e}")

print("\n--- Cell 4: Finished ---")


--- Cell 4: Fetching ALL Bank Transactions Per Account ---
   Token valid.

--- Processing Account: Tax Savings (0ba78d62-c570-4601-820b-0bb5a8464ef1) ---
   No bank transactions found for Tax Savings (page 1).
--- Finished Account: Tax Savings. Processed 0 total transactions. ---

--- Processing Account: Business Trans Acct (e961a9c8-27b9-4060-be6b-c9a8e860f0dc) ---
   No bank transactions found for Business Trans Acct (page 1).
--- Finished Account: Business Trans Acct. Processed 0 total transactions. ---

--- Processing Account: Business Trans Acct (Primary) (c1ea7488-36c7-41c8-a767-cf2f86a9d98e) ---
      Fetched 8 transactions on page 1.
   Reached last page for Business Trans Acct (Primary) (fetched 8).
--- Finished Account: Business Trans Acct (Primary). Processed 8 total transactions. ---

--- Processing Account: Business Income (03f6f8af-fa92-42af-be8c-e72df65f59bb) ---
      Fetched 2 transactions on page 1.
   Reached last page for Business Income (fetched 2).
--- Finished 

In [105]:
# Cell 5: Filter and Display UNRECONCILED Transactions

print("\n--- Cell 5: Filtering for UNRECONCILED Transactions ---")

# Initialize list for unreconciled items
unreconciled_transactions_list = []

# Check if the list from Cell 4 exists and is a list
if 'all_bank_transactions_list' in globals() and isinstance(all_bank_transactions_list, list):
    if not all_bank_transactions_list:
        print("   The list of fetched transactions from Cell 4 is empty. No filtering needed.")
    else:
        print(f"   Filtering {len(all_bank_transactions_list)} fetched transactions for 'UNRECONCILED' status...")
        # Iterate through the combined list from Cell 4
        for tx in all_bank_transactions_list:
            # Check the 'Status' field
            if tx.get('Status') == 'UNRECONCILED':
                unreconciled_transactions_list.append(tx)

        print(f"\n--- Found Total: {len(unreconciled_transactions_list)} UNRECONCILED Bank Transactions ---")

        # --- Print Details of UNRECONCILED Transactions ---
        if not unreconciled_transactions_list:
            print("\nNo transactions with status 'UNRECONCILED' were found.")
        else:
            # Optional: Sort the unreconciled list by date if desired
            # unreconciled_transactions_list.sort(key=lambda x: x.get('DateString', ''), reverse=True)

            print("\n--- DETAILS of UNRECONCILED Bank Transactions (JSON) ---")
            # Iterate through the filtered list
            for i, tx in enumerate(unreconciled_transactions_list):
                print(f"\n--- Unreconciled Tx {i+1} (Account: {tx.get('_account_name', 'N/A')}) ---")
                # Print full JSON
                print(json.dumps(tx, indent=4))
                # Or print specific fields:
                # print(f"    ID:       {tx.get('BankTransactionID')}")
                # print(f"    Date:     {tx.get('DateString')}")
                # print(f"    Type:     {tx.get('Type')}")
                # print(f"    Contact:  {tx.get('Contact', {}).get('Name', 'N/A')}")
                # print(f"    Total:    {tx.get('Total')}")
                # print(f"    Ref:      {tx.get('Reference', '')}")
                # print(f"    Status:   {tx.get('Status')}") # Should be UNRECONCILED

else:
    print("❌ ERROR: Variable 'all_bank_transactions_list' not found or is not a list.")
    print("   Please ensure Cell 4 (Fetch All Transactions) executed successfully before running this cell.")

print("\n--- Cell 5: Finished ---")


--- Cell 5: Filtering for UNRECONCILED Transactions ---
   Filtering 10 fetched transactions for 'UNRECONCILED' status...

--- Found Total: 0 UNRECONCILED Bank Transactions ---

No transactions with status 'UNRECONCILED' were found.

--- Cell 5: Finished ---
